<a href="https://colab.research.google.com/github/Azure06072005/DS317---Data-Mining-for-Enterprise/blob/main/reply_user_video_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%run '/content/drive/MyDrive/Colab Notebooks/MOOCubeX_dataset.ipynb'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (148 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to r

# Reply DataFrame

In [14]:
df_reply.show(30, truncate=True)

+-------------------+-----+-------------------------------------+----------+
|        create_time|   id|                                 text|   user_id|
+-------------------+-----+-------------------------------------+----------+
|2019-08-05 12:55:54| Rp_1|                             测试回复|U_10030806|
|2019-08-09 16:39:06| Rp_2|                                   赞|U_10031397|
|2019-08-10 22:39:35| Rp_3|                               好喜欢|U_10031531|
|2019-08-12 14:43:34| Rp_4|                             你也好棒|U_10031508|
|2019-08-12 14:44:51| Rp_5|                                 嗯对|U_10031508|
|2019-08-12 14:47:58| Rp_6|                           人工智能是|U_10031508|
|2019-08-13 09:41:32| Rp_7|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:42| Rp_8|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:53| Rp_9|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:59|Rp_10|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-19 18:08:53|Rp_11|                                11111|U_100

In [5]:
df_reply.printSchema()

root
 |-- create_time: string (nullable = true)
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [8]:
df_reply.describe()

DataFrame[summary: string, create_time: string, id: string, text: string, user_id: string]

In [18]:
from pyspark.sql import functions as F
miss = df_reply.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in df_reply.columns
])

miss.show()

+-----------+---+----+-------+
|create_time| id|text|user_id|
+-----------+---+----+-------+
|          0|  0|   0|      0|
+-----------+---+----+-------+

